# Markov-Chain Monte Carlo sampler
- the Gabor-GLM has a tractable likelihood
- the non-linear parametrization of the Gabor RF prohibits standard GLM methods for obtaining parameter estimates and posteriors
- the likelihood can still be used for MCMC sampling to obtain a (asysmptotically correct) posterior estimate

In [ ]:
%%capture
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np

import delfi.distribution as dd
import delfi.generator as dg
import delfi.inference as infer
import delfi.utils.io as io
from delfi.utils.viz import plot_pdf

from model.gabor_rf import maprf as model
from model.gabor_stats import maprfStats
from utils import setup_sim, setup_sampler, quick_plot, contour_draws, get_data_o

# parameters for this experiment

In [ ]:
seed = 42    # seed for generation of xo for selected cell. MCMC currently not seeded ! 

idx_cell = 6 # load toy cell number 6 (cosine-shaped RF with 1Hz firing rate)

fix_position=True         # fixues RF position during sampling to (0,0)
parametrization='logit_φ' # chosen parameterization of Gabor (affects priors !) 

n_samples = 1000000  # number of MCMC samples

savefile = 'results/MCMC/toycell_' + str(idx_cell) + '/maprf_MCMC_prior01_run_1_'+ str(n_samples)+'samples_param9_5min'

# load cell, generate xo

In [ ]:
# observation, models
reload_obs_stats = False

if reload_obs_stats:
    gtd = np.load('results/SNPE/toycell_6/ground_truth_data.npy', allow_pickle=True)[()]
    obs_stats = gtd['obs_stats']
    sim_info = np.load('results/sim_info.npy', allow_pickle=True)[()]
    d, params_ls = sim_info['d'], sim_info['params_ls']
    p = get_maprf_prior_01(params_ls)
    import delfi.generator as dg
    g = dg.Default(model=None, prior=p[0], summary=None)
else:
    import theano

    # result dirs
    !mkdir -p results/
    !mkdir -p results/SNPE/
    !mkdir -p results/SNPE/toycell_6/

    # training data and true parameters, data, statistics
    idx_cell = 6 # load toy cell number 6 (cosine-shaped RF with 1Hz firing rate)
    filename = 'results/toy_cells/toy_cell_' + str(idx_cell) + '.npy'

    g, prior, d = setup_sim(seed, path='')    
    g.model.rng = np.random.RandomState(seed=seed)
    params_dict_true = np.load(filename, allow_pickle=True)[()]
    params_dict_true['kernel']['t']['value'] = np.cast[theano.config.floatX](1.)
    g.model.params_dict = params_dict_true.copy()
    obs = g.model.gen_single()
    
    g, prior, d = setup_sim(seed, path='')    
    obs_stats, pars_true = get_data_o(filename, g, seed)
    
    assert np.all(g.summary.calc([obs]) == obs_stats)
    
    rf = g.model.params_to_rf(pars_true)[0]    
    
    # plot ground-truth receptive field
    plt.imshow(rf, interpolation='None')
    plt.show()
    obs_stats, obs_stats[0,-1] # summary statistics: (STA , spike count (over 5 minutes simulation) )

    np.save('results/SNPE/toycell_6/ground_truth_data',
            {'obs_stats' : obs_stats, 'pars_true' : pars_true, 'rf' : rf})

    # visualize RFs defined by prior-drawn parameters theta
    contour_draws(g.prior, g, obs_stats, d=d)
    print(obs_stats)

# set up MCMC sampling

In [ ]:
inference, data = setup_sampler(prior, obs, d, g, params_dict=params_dict_true, 
                          fix_position=False, parametrization='logit_φ')

inference.samplers[0].mu['logit_xo'] = prior['logit_xo']['mu'][0]
inference.samplers[0].mu['logit_yo'] = prior['logit_yo']['mu'][0]

inference.samplers[0].sd['logit_xo'] = prior['logit_xo']['sigma'][0]
inference.samplers[0].sd['logit_yo'] = prior['logit_yo']['sigma'][0]

# sample RF parameters (with Poisson bias marginalized out)

In [ ]:
T, L = inference.sample(n_samples)
T = {k.name: t for k, t in T.items()}

In [ ]:
# manually back-transform remaining parameters (location)
def scaled_expit_i(v):
    return 2. / (1. + np.exp(-v)) - 1

T['xo'], T['yo'] = scaled_expit_i(T['logit_xo']), scaled_expit_i(T['logit_yo'])

x,y = T['xo'], T['yo']

plt.figure(figsize=(15, 8))
plt.subplot(221)
plt.plot(x[0:])
plt.plot(y[0:])

plt.subplot(222)
plt.hist(x[0:], alpha=0.5, normed=True)

plt.subplot(224)
plt.hist(y[0:], alpha=0.5, normed=True)

plt.subplot(223)
plt.plot(x[0:], y[0:], '.k', alpha=0.1)
plt.show()

# sample Poisson bias (conditioned on the others)
- above sampler did not sample biases, but instead marginalized them out during sampling (much faster mixing)
- conditionally on all other parameters, we can still sample the biases
- note this only works for an exponential distribution as prior. 

In [ ]:
inference.sample_biases(data, T, g.model.dt)

plt.figure(figsize=(12,5))
plt.subplot(2,1,1)
plt.plot(T['bias'])
print('mean: ' + str(T['bias'].mean()) + ', var: ' + str(T['bias'].var()))
plt.subplot(2,1,2)
plt.plot(T['λo'])
print('mean: ' + str(T['λo'].mean()) + ', var: ' + str(T['λo'].var()))
plt.show()

# example posterior draws (in direct comparison with xo)

In [ ]:

plt.figure(figsize=(16,12))
i = 1
for t in np.sort(np.random.choice(T['gain'].shape[0], 12, replace=False)):
    params_dict = {'kernel' : {'s' : {}, 'l' : {}}, 'glm': {}}
    params_dict['glm']['bias'] = T['bias'][t]
    params_dict['kernel']['s']['phase'] = T['phase'][t]
    params_dict['kernel']['s']['angle'] = T['angle'][t] 
    params_dict['kernel']['s']['freq']  = T['freq'][t]
    params_dict['kernel']['s']['ratio'] = T['ratio'][t]
    params_dict['kernel']['s']['width'] = T['width'][t]
    params_dict['kernel']['s']['gain'] = T['gain'][t]
    params_dict['kernel']['l']['xo'] = T['xo'][t]
    params_dict['kernel']['l']['yo'] = T['yo'][t]

    axis_x = g.model.axis_x - params_dict['kernel']['l']['xo']
    axis_y = g.model.axis_y - params_dict['kernel']['l']['yo']    
    g.model._gen.grid_x, g.model._gen.grid_y = np.meshgrid(axis_x, axis_y)    
    
    ks = g.model._eval_ks(bias=params_dict['glm']['bias'], 
                    angle=params_dict['kernel']['s']['angle'],
                    freq=params_dict['kernel']['s']['freq'],
                    gain=params_dict['kernel']['s']['gain'],
                    phase=params_dict['kernel']['s']['phase'],
                    ratio=params_dict['kernel']['s']['ratio'],
                    width=params_dict['kernel']['s']['width'])
    
    plt.subplot(3,4,i)
    plt.imshow(np.hstack((ks.reshape(d,d), g.model.params_to_rf(pars_true)[0])), interpolation='None')
    plt.title('t =' + str(t))
    
    print('loc:' , [T['xo'][t], T['yo'][t]])    
    i += 1
    
plt.show()

# marginal histograms for each (transformed) parameter

In [ ]:
burnin = 50

for key in ['bias', 'λo', 
            'gain', 'log_A', 'phase', 'logit_φ',
            'angle', 'logit_θ', 'freq', 'log_f',
            'ratio', 'width', 'log_γ', 'log_b', 
            'xo', 'yo', 'logit_xo', 'logit_yo'
            ]:
    
    if key in T.keys():
        x = T[key][burnin:]
        plt.hist(x, bins=np.linspace(x.min(), x.max(), 20), alpha=0.5, normed=True)
        plt.title(key)
        plt.show()
        print('mean:', x.mean())
        print('var:', x.var())

# posterior samples versus prior


## actual parameters

In [ ]:
samples = np.hstack([np.atleast_2d(T[key].T).T for key in ['bias', 'gain', 'phase', 'freq','angle','ratio','width', 'xo', 'yo']])

pars_raw = np.array([ params_dict_true['glm']['bias'],
                      params_dict_true['kernel']['s']['gain'],
                      params_dict_true['kernel']['s']['phase'],
                      params_dict_true['kernel']['s']['freq'],
                      params_dict_true['kernel']['s']['angle'],
                      params_dict_true['kernel']['s']['ratio'],
                      params_dict_true['kernel']['s']['width'],
                      params_dict_true['kernel']['l']['xo'],
                      params_dict_true['kernel']['l']['yo'] ])

plot_pdf(g.prior, lims=[-3,3], gt=pars_raw.reshape(-1), figsize=(16,16), resolution=100, samples=samples.T,
         ticks=True, labels_params=['bias', 'gain', 'phase', 'freq', 'angle', 'ratio', 'width', 'xo', 'yo']);

## parameters in log/logit space

In [ ]:
samples = np.hstack([np.atleast_2d(T[key].T).T for key in ['bias', 'log_A', 'logit_φ', 'log_f','logit_θ','log_γ','log_b', 'logit_xo', 'logit_yo']])

plot_pdf(g.prior, lims=[-3,3], gt=pars_true.reshape(-1), figsize=(16,16), resolution=100, samples=samples.T,
         ticks=True, labels_params=['bias', 'gain', 'logit phase', 'log freq', 'logit angle', 'log ratio', 'log width', 'logit xo', 'logit yo']);

# (roughly) check for mixing of the chain
- the MCMC sampler will have trouble to mix between the different distinct posterior modes
- since we know the symmetries of the Gabor parametrization that cause the multimodality, we can later just 'mirror' the samples to obtain all posterior modes

In [ ]:
plt.figure(figsize = (16,5) )
plt.plot(samples)
plt.show()

# save results

In [ ]:
savefile

In [ ]:
np.save(savefile, {'T' : T, 'params_dict_true' : params_dict_true})